<a href="https://colab.research.google.com/github/vannguyen799/CloudSync-GoogleColab/blob/main/CopyDriveFolderAutomation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hướng dẫn sử dụng Công cụ đồng bộ hóa Google Drive

Công cụ này giúp bạn sao chép tệp và thư mục từ một thư mục Google Drive nguồn sang một thư mục đích.

## Các bước thực hiện:

1.  **Chạy tất cả các ô:** Nhấp vào "Runtime" (Thời gian chạy) trên menu, sau đó chọn "Run all" (Chạy tất cả).
2.  **Giao diện chọn thư mục:** Một giao diện sẽ xuất hiện ở đầu notebook cho phép bạn nhập ID thư mục nguồn và đích.
3.  **Lấy ID thư mục Google Drive:**
    *   Mở Google Drive trên trình duyệt web của bạn.
    *   Điều hướng đến thư mục bạn muốn sử dụng làm nguồn hoặc đích.
    *   Nhìn vào thanh địa chỉ của trình duyệt. ID thư mục là chuỗi ký tự sau `/folders/` trong URL.
        Ví dụ: Trong URL `https://drive.google.com/drive/folders/YOUR_FOLDER_ID`, `YOUR_FOLDER_ID` chính là ID thư mục.
    *   Sao chép ID này.
4.  **Nhập ID vào giao diện:** Dán ID thư mục nguồn và đích vào các ô văn bản tương ứng trong giao diện.
5.  **Thiết lập ID:** Nhấp vào nút "Set Folder IDs". Bạn sẽ thấy thông báo xác nhận ID đã được đặt.
6.  **Chạy đồng bộ hóa:** Nhấp vào nút "Run Synchronization". Chương trình sẽ bắt đầu đọc cấu trúc thư mục, so sánh và sao chép các tệp cần thiết.
7.  **Theo dõi tiến trình:** Xem đầu ra bên dưới ô mã để theo dõi tiến trình đọc cấu trúc, so sánh và sao chép.

## Lưu ý:

*   **Cấp quyền truy cập:** Lần đầu chạy, bạn sẽ được yêu cầu cấp quyền truy cập vào Google Drive. Hãy chắc chắn chấp nhận các yêu cầu này để công cụ có thể đọc và ghi tệp.
*   **Bộ nhớ:** Đối với các thư mục có số lượng tệp rất lớn, quá trình đọc cấu trúc và so sánh có thể yêu cầu lượng bộ nhớ đáng kể. Nếu gặp lỗi bộ nhớ, bạn có thể cần chia nhỏ thao tác hoặc sử dụng phiên bản Colab có cấu hình mạnh hơn.
*   Tùy thuộc vào kích thước và số lượng tệp trong thư mục của bạn, quá trình này có thể mất một chút thời gian.
*   Các tệp của Google (như Google Docs, Sheets, Slides) sẽ được sao chép lại thành các tệp mới, không phải là bản sao tham chiếu.

# V3 - METADATA FIRST

In [1]:
# @title main

from google.colab import widgets
import ipywidgets as ipw
from IPython.display import display, HTML
from google.colab import drive
import os

# Import libraries for sync functionality
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import threading
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
import hashlib
from collections import defaultdict
import json
import os

# Authenticate user
auth.authenticate_user()

# Display custom CSS for dark mode styling
display(HTML("""
<style>
/* Dark mode theme - Full width optimized */
.widget-container {
    max-width: 100%;
    margin: 0;
    padding: 10px;
    background-color: #1a1a1a;
    border-radius: 6px;
    box-shadow: 0 2px 10px rgba(0,0,0,0.2);
}

.widget-title {
    text-align: center;
    font-size: 22px;
    font-weight: bold;
    margin-bottom: 10px;
    padding: 10px;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    border-radius: 6px;
    box-shadow: 0 1px 6px rgba(102, 126, 234, 0.2);
}

.widget-section {
    margin: 4px 0;
    padding: 8px;
    background-color: #2d2d2d;
    border-radius: 6px;
    border-left: 3px solid #667eea;
    box-shadow: 0 1px 3px rgba(0,0,0,0.1);
}

.widget-section-title {
    font-weight: bold;
    color: #e0e0e0;
    margin-bottom: 6px;
    font-size: 15px;
}

/* Dark mode for widgets */
.widget-text input, .widget-dropdown select, .widget-text input[type="number"] {
    background-color: #3d3d3d !important;
    color: #e0e0e0 !important;
    border: 1px solid #555 !important;
    border-radius: 4px !important;
}

.widget-label {
    color: #e0e0e0 !important;
}

/* Hide scrollbars and fix widget layout */
.widget-box {
    overflow: hidden !important;
}

.widget-hbox, .widget-vbox {
    overflow: hidden !important;
    align-items: center !important;
}

/* Compact spacing */
.compact-row {
    display: flex;
    justify-content: space-between;
    align-items: center;
    gap: 8px;
    margin: 4px 0;
}

/* Enhanced Run Button Styling - Green Theme */
.widget-button.btn-primary, button.btn-primary, .widget-button.btn-success, button.btn-success {
    background: linear-gradient(135deg, #4CAF50 0%, #45a049 100%) !important;
    border: none !important;
    border-radius: 6px !important;
    font-weight: bold !important;
    font-size: 14px !important;
    text-shadow: 0 1px 2px rgba(0,0,0,0.3) !important;
    box-shadow: 0 1px 6px rgba(76, 175, 80, 0.3) !important;
    transition: all 0.3s ease !important;
    color: white !important;
}

.widget-button.btn-primary:hover, button.btn-primary:hover, .widget-button.btn-success:hover, button.btn-success:hover {
    background: linear-gradient(135deg, #45a049 0%, #4CAF50 100%) !important;
    box-shadow: 0 6px 20px rgba(76, 175, 80, 0.6) !important;
    transform: translateY(-2px) !important;
}

.widget-button.btn-primary:active, button.btn-primary:active, .widget-button.btn-success:active, button.btn-success:active {
    transform: translateY(0px) !important;
    box-shadow: 0 2px 10px rgba(76, 175, 80, 0.4) !important;
}

/* Minimize Button Styling */
.widget-button:not(.btn-primary) {
    background-color: #444 !important;
    border: 1px solid #666 !important;
    border-radius: 4px !important;
    color: #ccc !important;
    font-weight: bold !important;
    font-size: 16px !important;
    transition: all 0.2s ease !important;
}

.widget-button:not(.btn-primary):hover {
    background-color: #555 !important;
    border-color: #777 !important;
    color: white !important;
}

/* Force Green Color for Run Button - Multiple Selectors */
button[description*="RUN SYNC"], .widget-button[description*="RUN SYNC"],
.widget-button.btn-success, button.btn-success,
.jupyter-button.btn-success, .jupyter-widgets.btn-success {
    background: linear-gradient(135deg, #4CAF50 0%, #45a049 100%) !important;
    border: none !important;
    border-radius: 6px !important;
    font-weight: bold !important;
    font-size: 14px !important;
    text-shadow: 0 1px 2px rgba(0,0,0,0.3) !important;
    box-shadow: 0 1px 6px rgba(76, 175, 80, 0.3) !important;
    transition: all 0.3s ease !important;
    color: white !important;
}

button[description*="RUN SYNC"]:hover, .widget-button[description*="RUN SYNC"]:hover,
.widget-button.btn-success:hover, button.btn-success:hover,
.jupyter-button.btn-success:hover, .jupyter-widgets.btn-success:hover {
    background: linear-gradient(135deg, #45a049 0%, #4CAF50 100%) !important;
    box-shadow: 0 6px 20px rgba(76, 175, 80, 0.6) !important;
    transform: translateY(-2px) !important;
}

/* Additional specific targeting for ipywidgets */
.widget-area .widget-button.btn-success,
.widget-area button.btn-success,
.output_area .widget-button.btn-success,
.output_area button.btn-success {
    background: linear-gradient(135deg, #4CAF50 0%, #45a049 100%) !important;
    border: none !important;
    color: white !important;
}
</style>
"""))

# Global variables to store configurations
source_folder_id = ''
destination_folder_id = ''
comparison_mode = 'name_only'
num_threads = 4
retry_max_attempts = 5
retry_base_delay = 1.0
retry_max_delay = 60
retry_backoff_factor = 2.0
retry_jitter = True

# Global variables to control sync process
sync_running = False
stop_requested = False
sync_thread = None

# Drive service
drive_service = build('drive', 'v3')

# Global variable to store original print function
original_print = print

# Custom print function that redirects to config_output
def custom_print(*args, **kwargs):
    """Custom print function that outputs only to config_output widget"""
    try:
        if 'config_output' in globals() and config_output is not None:
            # Handle common print parameters
            sep = kwargs.get('sep', ' ')
            end = kwargs.get('end', '\n')
            file = kwargs.get('file', None)
            flush = kwargs.get('flush', False)

            # Convert all arguments to strings and join them
            if args:
                message = sep.join(str(arg) for arg in args)
            else:
                message = ''

            # Add the message to config_output using original print to avoid recursion
            with config_output:
                original_print(message, end=end, flush=flush)
        else:
            # Fallback to original print if config_output is not available
            original_print(*args, **kwargs)
    except Exception as e:
        # Fallback to original print if there's any error
        original_print(f"Error in custom_print: {e}")
        original_print(*args, **kwargs)

# Function to enable/disable print redirection
def enable_print_redirection():
    """Enable print redirection to config_output"""
    import builtins
    builtins.print = custom_print

def disable_print_redirection():
    """Disable print redirection and restore original print"""
    import builtins
    builtins.print = original_print

# Configuration for retry logic
RETRY_CONFIG = {
    'max_attempts': 5,
    'base_delay': 1.0,
    'max_delay': 60,
    'backoff_factor': 2.0,
    'jitter': True
}

# Function to extract folder ID from Google Drive link
def extract_folder_id_from_link(input_text):
    """Extract folder ID from Google Drive link or return the input if it's already an ID"""
    import re

    input_text = input_text.strip()

    # If input is empty, return empty
    if not input_text:
        return input_text

    # Check if input is already a folder ID (alphanumeric string, typically 25-50 chars)
    # Google Drive folder IDs are usually 28-44 characters long
    if re.match(r'^[a-zA-Z0-9_-]{20,50}$', input_text) and 'drive.google.com' not in input_text:
        return input_text

    # Extract folder ID from various Google Drive URL formats
    patterns = [
        # Standard sharing link: https://drive.google.com/drive/folders/FOLDER_ID
        r'drive\.google\.com/drive/folders/([a-zA-Z0-9_-]+)',
        # Open link: https://drive.google.com/drive/u/0/folders/FOLDER_ID
        r'drive\.google\.com/drive/u/\d+/folders/([a-zA-Z0-9_-]+)',
        # Folder view link: https://drive.google.com/drive/folders/FOLDER_ID?usp=sharing
        r'drive\.google\.com/drive/folders/([a-zA-Z0-9_-]+)\?',
        # Alternative format: https://drive.google.com/folderview?id=FOLDER_ID
        r'drive\.google\.com/folderview\?id=([a-zA-Z0-9_-]+)',
        # Another format: https://drive.google.com/open?id=FOLDER_ID
        r'drive\.google\.com/open\?id=([a-zA-Z0-9_-]+)',
    ]

    for pattern in patterns:
        match = re.search(pattern, input_text)
        if match:
            return match.group(1)

    # If no pattern matches but it contains drive.google.com, it might be a malformed URL
    if 'drive.google.com' in input_text:
        print(f"⚠️  Warning: Could not extract folder ID from URL: {input_text}")
        print("💡 Please check the URL format or paste the folder ID directly.")
        return input_text  # Return original input for user to see the issue

    # If it doesn't contain drive.google.com, assume it's a folder ID
    return input_text

# Get user email for personalized config
def get_user_email():
    """Get current user's email"""
    try:
        from google.colab import auth
        import google.auth
        from google.auth.transport.requests import Request
        import requests

        # Get credentials
        creds, _ = google.auth.default()
        creds.refresh(Request())

        # Get user info
        response = requests.get(
            'https://www.googleapis.com/oauth2/v1/userinfo',
            headers={'Authorization': f'Bearer {creds.token}'}
        )

        if response.status_code == 200:
            user_info = response.json()
            return user_info.get('email', 'unknown_user')
        else:
            return 'unknown_user'
    except Exception as e:
        print(f"⚠️  Could not get user email: {e}")
        return 'unknown_user'

# Configuration file path (personalized by user email)
def get_config_file_path():
    """Get config file path for current user"""
    user_email = get_user_email()
    # Replace @ and . with _ for safe filename
    safe_email = user_email.replace('@', '_').replace('.', '_')

    # Create config directory if it doesn't exist
    config_dir = '/content/drive/MyDrive/Colab Notebooks/.config'
    if not os.path.exists(config_dir):
        os.makedirs(config_dir, exist_ok=True)

    return f'{config_dir}/sync_tool_config_{safe_email}.json'

def save_config():
    """Save current configuration to file"""
    try:
        # Mount Google Drive if not already mounted
        if not os.path.exists('/content/drive'):
            drive.mount('/content/drive')

        config_file = get_config_file_path()
        user_email = get_user_email()

        # Convert source folder link to ID if needed before saving
        source_input = src_picker.value.strip()
        source_folder_id_to_save = extract_folder_id_from_link(source_input)

        config = {
            'user_email': user_email,
            'saved_at': datetime.now().isoformat(),
            'source_folder_id': source_folder_id_to_save,
            'destination_folder_id': dst_picker.value.strip(),
            'comparison_mode': comparison_mode_selector.value,
            'num_threads': num_threads_input.value,
            'retry_max_attempts': retry_max_attempts_input.value,
            'retry_base_delay': retry_base_delay_input.value,
            'retry_max_delay': retry_max_delay_input.value,
            'retry_backoff_factor': retry_backoff_factor_input.value,
            'retry_jitter': retry_jitter_checkbox.value,
            'redirect_print': redirect_print_checkbox.value
        }

        with open(config_file, 'w') as f:
            json.dump(config, f, indent=2)

        return config_file
    except Exception as e:
        print(f"❌ Error saving config: {e}")
        return False

def load_config():
    """Load configuration from file"""
    try:
        # Mount Google Drive if not already mounted
        if not os.path.exists('/content/drive'):
            drive.mount('/content/drive')

        config_file = get_config_file_path()
        current_user = get_user_email()

        if os.path.exists(config_file):
            with open(config_file, 'r') as f:
                config = json.load(f)

            # Verify config belongs to current user
            saved_user = config.get('user_email', 'unknown')
            if saved_user != current_user:
                print(f"⚠️  Config file belongs to different user: {saved_user}")
                return False

            # Apply loaded config to widgets
            src_picker.value = config.get('source_folder_id', '')
            dst_picker.value = config.get('destination_folder_id', '')
            comparison_mode_selector.value = config.get('comparison_mode', 'name_only')
            num_threads_input.value = config.get('num_threads', 4)
            retry_max_attempts_input.value = config.get('retry_max_attempts', 5)
            retry_base_delay_input.value = config.get('retry_base_delay', 1.0)
            retry_max_delay_input.value = config.get('retry_max_delay', 60)
            retry_backoff_factor_input.value = config.get('retry_backoff_factor', 2.0)
            retry_jitter_checkbox.value = config.get('retry_jitter', True)
            redirect_print_checkbox.value = config.get('redirect_print', True)

            return config_file
        else:
            return False
    except Exception as e:
        print(f"⚠️  No saved config found or error loading: {e}")
        return False

# Create main container with dark theme - Full width
main_container = ipw.VBox(layout=ipw.Layout(
    width='100%',
    margin='0',
    padding='10px',
    border='1px solid #444',
    border_radius='6px',
    background_color='#1a1a1a'
))

# Title with Run button in header
title_text = ipw.HTML(
    value='<div class="widget-title">🚀 Google Drive Folder Sync Tool</div>',
    layout=ipw.Layout(width='70%', height='50px')
)

# Run button in header with enhanced styling - Blue color
header_run_btn = ipw.Button(
    description="🚀 RUN SYNC",
    button_style='primary',
    layout=ipw.Layout(
        width='18%',
        height='50px',
        border_radius='6px'
    )
)

# Stop button in header with red styling
header_stop_btn = ipw.Button(
    description="🛑 STOP",
    button_style='danger',
    layout=ipw.Layout(
        width='8%',
        height='50px',
        border_radius='6px'
    ),
    disabled=True
)

# Force blue color with JavaScript after button creation
display(HTML("""
<script>
setTimeout(function() {
    // Find all buttons with "RUN SYNC" text and force blue styling
    var buttons = document.querySelectorAll('button');
    buttons.forEach(function(btn) {
        if (btn.textContent.includes('RUN SYNC')) {
            btn.style.background = 'linear-gradient(135deg, #2196F3 0%, #1976D2 100%)';
            btn.style.border = 'none';
            btn.style.color = 'white';
            btn.style.fontWeight = 'bold';
            btn.style.borderRadius = '6px';
            btn.style.boxShadow = '0 1px 6px rgba(33, 150, 243, 0.3)';

            // Add hover effect
            btn.addEventListener('mouseenter', function() {
                this.style.background = 'linear-gradient(135deg, #1976D2 0%, #2196F3 100%)';
                this.style.boxShadow = '0 6px 20px rgba(33, 150, 243, 0.6)';
                this.style.transform = 'translateY(-2px)';
            });

            btn.addEventListener('mouseleave', function() {
                this.style.background = 'linear-gradient(135deg, #2196F3 0%, #1976D2 100%)';
                this.style.boxShadow = '0 1px 6px rgba(33, 150, 243, 0.3)';
                this.style.transform = 'translateY(0px)';
            });
        }

        // Style STOP buttons with red gradient
        if (btn.textContent.includes('STOP')) {
            btn.style.background = 'linear-gradient(135deg, #f44336 0%, #d32f2f 100%)';
            btn.style.border = 'none';
            btn.style.color = 'white';
            btn.style.fontWeight = 'bold';
            btn.style.borderRadius = '6px';
            btn.style.boxShadow = '0 1px 6px rgba(244, 67, 54, 0.3)';

            // Add hover effect for stop button
            btn.addEventListener('mouseenter', function() {
                if (!this.disabled) {
                    this.style.background = 'linear-gradient(135deg, #d32f2f 0%, #f44336 100%)';
                    this.style.boxShadow = '0 6px 20px rgba(244, 67, 54, 0.6)';
                    this.style.transform = 'translateY(-2px)';
                }
            });

            btn.addEventListener('mouseleave', function() {
                if (!this.disabled) {
                    this.style.background = 'linear-gradient(135deg, #f44336 0%, #d32f2f 100%)';
                    this.style.boxShadow = '0 1px 6px rgba(244, 67, 54, 0.3)';
                    this.style.transform = 'translateY(0px)';
                }
            });
        }
    });
}, 1000);
</script>
"""))

# Header container with title and buttons - Perfect alignment
title = ipw.HBox([
    title_text,
    header_run_btn,
    header_stop_btn
], layout=ipw.Layout(
    width='100%',
    justify_content='space-between',
    align_items='center',
    margin='0 0 10px 0',
    gap='1%'
))

def pick_drive_folder(title="Select a folder", is_source=False):
    # Mount Google Drive if not already mounted
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')

    # Different placeholder text for source folder (supports links)
    if is_source:
        placeholder_text = 'Paste Google Drive folder link or ID here'
    else:
        placeholder_text = 'Paste Google Drive folder ID here'

    folder_picker = ipw.Text(
        value='',
        placeholder=placeholder_text,
        description=title,
        style={'description_width': '150px'},
        layout=ipw.Layout(width='100%')
    )
    return folder_picker

# Folder selection section
folder_section_title = ipw.HTML(value='<div class="widget-section-title">📁 Folder Selection</div>')
src_picker = pick_drive_folder("Source:", is_source=True)
dst_picker = pick_drive_folder("Destination:")

folder_section = ipw.VBox([
    folder_section_title,
    src_picker,
    dst_picker
], layout=ipw.Layout(
    width='49%',
    # margin='2px',
    padding='8px',
    border='1px solid #444',
    border_radius='6px',
    background_color='#2d2d2d',
    min_height='120px'
))

# Comparison mode section
comparison_section_title = ipw.HTML(value='<div class="widget-section-title">🔍 Comparison Settings</div>')
comparison_mode_selector = ipw.Dropdown(
    options=[
        ('Name only', 'name_only'),
        ('Name + Size', 'name_and_size'),
        ('Name + Size + Date', 'name_size_date'),
        ('Name + Checksum (slow)', 'checksum')
    ],
    value='name_only',
    description='Mode:',
    style={'description_width': '80px'},
    layout=ipw.Layout(width='100%')
)

comparison_section = ipw.VBox([
    comparison_section_title,
    comparison_mode_selector
], layout=ipw.Layout(
    width='49%',
    # margin='2px',
    padding='8px',
    border='1px solid #444',
    border_radius='6px',
    background_color='#2d2d2d',
    min_height='120px'
))

# Performance settings section - 3 boxes in single row with minimize button
performance_minimize_btn = ipw.Button(
    description="−",
    button_style='',
    layout=ipw.Layout(width='30px', height='25px'),
    tooltip='Minimize/Expand Performance Settings'
)

# Config save/load buttons
save_config_btn = ipw.Button(
    description="💾 Save Config",
    button_style='info',
    layout=ipw.Layout(width='100px', height='25px'),
    tooltip='Save current configuration'
)

load_config_btn = ipw.Button(
    description="📂 Load Config",
    button_style='warning',
    layout=ipw.Layout(width='100px', height='25px'),
    tooltip='Load saved configuration'
)

performance_section_title = ipw.HBox([
    ipw.HTML(value='<div class="widget-section-title">⚡ Performance</div>', layout=ipw.Layout(width='70%')),
    # save_config_btn,
    # load_config_btn,
    performance_minimize_btn
], layout=ipw.Layout(width='100%', justify_content='space-between', align_items='center', gap='5px'))

# Box 1: Threading & Retry Count
num_threads_input = ipw.IntText(
    value=4,
    description='Threads:',
    disabled=False,
    style={'description_width': '60px'},
    layout=ipw.Layout(width='100%')
)

retry_max_attempts_input = ipw.IntText(
    value=5,
    description='Retries:',
    disabled=False,
    style={'description_width': '60px'},
    layout=ipw.Layout(width='100%')
)

performance_box1 = ipw.VBox([num_threads_input, retry_max_attempts_input], layout=ipw.Layout(
    width='32%',
    margin='1px',
    padding='6px',
    border='1px solid #555',
    border_radius='4px',
    background_color='#3d3d3d'
))

# Box 2: Delay Settings
retry_base_delay_input = ipw.FloatText(
    value=1.0,
    description='Base Delay:',
    disabled=False,
    style={'description_width': '70px'},
    layout=ipw.Layout(width='100%')
)

retry_max_delay_input = ipw.IntText(
    value=60,
    description='Max Delay:',
    disabled=False,
    style={'description_width': '70px'},
    layout=ipw.Layout(width='100%')
)

performance_box2 = ipw.VBox([retry_base_delay_input, retry_max_delay_input], layout=ipw.Layout(
    width='32%',
    margin='1px',
    padding='6px',
    border='1px solid #555',
    border_radius='4px',
    background_color='#3d3d3d'
))

# Box 3: Backoff & Jitter
retry_backoff_factor_input = ipw.FloatText(
    value=2.0,
    description='Backoff:',
    disabled=False,
    style={'description_width': '60px'},
    layout=ipw.Layout(width='100%')
)

retry_jitter_checkbox = ipw.Checkbox(
    value=True,
    description='Jitter',
    style={'description_width': '50px'},
    layout=ipw.Layout(width='100%')
)

# Add checkbox for print redirection
redirect_print_checkbox = ipw.Checkbox(
    value=True,
    description='Redirect Logs',
    style={'description_width': '80px'},
    layout=ipw.Layout(width='100%'),
    tooltip='Redirect all print statements to Status & Output section'
)

performance_box3 = ipw.VBox([retry_backoff_factor_input, retry_jitter_checkbox, redirect_print_checkbox], layout=ipw.Layout(
    width='32%',
    margin='1px',
    padding='6px',
    border='1px solid #555',
    border_radius='4px',
    background_color='#3d3d3d'
))

# Performance content container
performance_content = ipw.HBox([performance_box1, performance_box2, performance_box3], layout=ipw.Layout(
    width='100%',
    justify_content='space-between',
    align_items='flex-start'
))

# Performance section with 3 boxes in single row
performance_section = ipw.VBox([
    performance_section_title,
    performance_content
], layout=ipw.Layout(
    width='100%',
    margin='2px',
    padding='8px',
    border='1px solid #444',
    border_radius='6px',
    background_color='#2d2d2d',
    min_height='120px'
))

# Minimize/Expand functionality for Performance section
def toggle_performance_section(_):
    if performance_content.layout.display == 'none':
        # Expand
        performance_content.layout.display = ''
        performance_minimize_btn.description = '−'
        performance_section.layout.min_height = '120px'
    else:
        # Minimize
        performance_content.layout.display = 'none'
        performance_minimize_btn.description = '+'
        performance_section.layout.min_height = '50px'

performance_minimize_btn.on_click(toggle_performance_section)

# Config button handlers
def save_config_handler(_):
    """Handle save config button click"""
    result = save_config()
    if result:
        user_email = get_user_email()
        with config_output:
            print("✅ Configuration saved successfully!")
            print(f"� User: {user_email}")
            print(f"�� Saved to: {result}")
    else:
        with config_output:
            print("❌ Failed to save configuration!")

def load_config_handler(_):
    """Handle load config button click"""
    result = load_config()
    if result:
        user_email = get_user_email()
        with config_output:
            print("✅ Configuration loaded successfully!")
            print(f"��User: {user_email}")
            print(f"�� Loaded from: {result}")
    else:
        with config_output:
            print("⚠️  No saved configuration found or failed to load!")

save_config_btn.on_click(save_config_handler)
load_config_btn.on_click(load_config_handler)

# ============================================================================
# SYNC FUNCTIONALITY CLASSES AND FUNCTIONS
# ============================================================================

class FileMetadata:
    """Class to store file metadata"""
    def __init__(self, file_data):
        self.id = file_data.get('id')
        self.name = file_data.get('name')
        self.size = file_data.get('size', '0')
        self.modified_time = file_data.get('modifiedTime')
        self.mime_type = file_data.get('mimeType')
        self.md5_checksum = file_data.get('md5Checksum')
        self.parents = file_data.get('parents', [])
        self.is_folder = self.mime_type == 'application/vnd.google-apps.folder'

    def __repr__(self):
        return f"FileMetadata(name='{self.name}', size={self.size}, is_folder={self.is_folder})"

def format_size(size_bytes):
    """Format size in bytes to human readable format"""
    if not size_bytes or size_bytes == 0:
        return "0 B"

    try:
        size_bytes = int(size_bytes)
    except:
        return "Unknown"

    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size_bytes < 1024.0:
            return f"{size_bytes:.1f} {unit}"
        size_bytes /= 1024.0
    return f"{size_bytes:.1f} PB"

def stop_sync(_):
    """Stop the synchronization process"""
    global stop_requested, sync_running, sync_thread

    # Set stop flag
    stop_requested = True

    # Disable stop button immediately to prevent multiple clicks
    header_stop_btn.disabled = True
    header_stop_btn.description = "🔄 Stopping..."

    config_output.clear_output()
    with config_output:
        original_print("🛑 STOP requested by user...")
        original_print("⏳ Waiting for current operations to complete safely...")

    # Create a separate thread to handle the stopping process
    def stop_worker():
        global sync_running

        # If thread is running, try to join it with timeout
        if sync_thread and sync_thread.is_alive():
            try:
                with config_output:
                    original_print("⏳ Waiting for sync thread to finish...")

                # Wait up to 10 seconds for graceful shutdown
                sync_thread.join(timeout=10.0)

                if sync_thread.is_alive():
                    with config_output:
                        original_print("⚠️  Thread still running after 10 seconds...")
                        original_print("🔄 Continuing to wait for graceful shutdown...")

                    # Wait another 10 seconds
                    sync_thread.join(timeout=10.0)

                    if sync_thread.is_alive():
                        with config_output:
                            original_print("⚠️  Thread still running after 20 seconds - this is unusual")
                            original_print("💡 The sync process may be stuck in a long operation")
                            original_print("🔄 Will continue waiting...")

            except Exception as e:
                with config_output:
                    original_print(f"⚠️  Error while stopping thread: {e}")

        # Update final state
        sync_running = False

        # Reset button states
        header_run_btn.disabled = False
        header_run_btn.description = "🚀 RUN SYNC"
        header_stop_btn.disabled = True
        header_stop_btn.description = "🛑 STOP"

        with config_output:
            original_print("✅ Synchronization stopped successfully.")
            original_print("🔄 Ready for next sync operation.")

    # Start stop worker in a separate thread to avoid blocking UI
    stop_thread = threading.Thread(target=stop_worker, daemon=True)
    stop_thread.start()

def run_sync(_):
    # Auto-save configuration first
    global source_folder_id, destination_folder_id, comparison_mode, num_threads, \
           retry_max_attempts, retry_base_delay, retry_max_delay, retry_backoff_factor, retry_jitter, \
           sync_running, stop_requested, sync_thread

    # Reset stop flag and enable/disable buttons
    stop_requested = False
    sync_running = True
    header_run_btn.disabled = True
    header_run_btn.description = "🔄 Starting..."
    header_stop_btn.disabled = False

    # Get source folder input and convert link to ID if needed
    source_folder_input = src_picker.value.strip()
    source_folder_id = extract_folder_id_from_link(source_folder_input)

    destination_folder_id = dst_picker.value.strip()
    comparison_mode = comparison_mode_selector.value
    num_threads = num_threads_input.value
    retry_max_attempts = retry_max_attempts_input.value
    retry_base_delay = retry_base_delay_input.value
    retry_max_delay = retry_max_delay_input.value
    retry_backoff_factor = retry_backoff_factor_input.value
    retry_jitter = retry_jitter_checkbox.value

    # Show conversion result if a link was converted
    if source_folder_input != source_folder_id and source_folder_input:
        config_output.clear_output()
        with config_output:
            print(f"🔗 Link converted to folder ID:")
            print(f"   Original: {source_folder_input[:50]}{'...' if len(source_folder_input) > 50 else ''}")
            print(f"   Folder ID: {source_folder_id}")
            print()

    # Update RETRY_CONFIG with values from UI
    RETRY_CONFIG['max_attempts'] = retry_max_attempts
    RETRY_CONFIG['base_delay'] = retry_base_delay
    RETRY_CONFIG['max_delay'] = retry_max_delay
    RETRY_CONFIG['backoff_factor'] = retry_backoff_factor
    RETRY_CONFIG['jitter'] = retry_jitter

    # Validate source folder ID - required
    if not source_folder_id:
        config_output.clear_output()
        with config_output:
            print("⚠️  ERROR: Source folder ID is required!")
            print("=" * 50)
            print("❌ Please provide a valid Google Drive source folder ID or link.")
            print("💡 You can paste either:")
            print("   • Folder ID: 1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms")
            print("   • Folder Link: https://drive.google.com/drive/folders/1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms")
            print("=" * 50)

        # Reset button states
        sync_running = False
        header_run_btn.disabled = False
        header_stop_btn.disabled = True
        return

    # Handle destination folder ID - default to root if empty
    if not destination_folder_id:
        destination_folder_id = 'root'
        config_output.clear_output()
        with config_output:
            print("⚠️  Destination folder ID is empty. Defaulting to Google Drive root folder.")
            print("💡 Files will be copied to the root of your Google Drive.")

    # Auto-save configuration before starting sync
    if save_config():
        config_saved_msg = "✅ Configuration Auto-Saved"
    else:
        config_saved_msg = "⚠️  Configuration save failed"

    config_output.clear_output()
    with config_output:
        print(f"{config_saved_msg} & Starting Sync...")
        print("=" * 50)
        print(f"📂 Source: {source_folder_id}")
        print(f"📂 Destination: {destination_folder_id}")
        print(f"🔍 Mode: {comparison_mode}")
        print(f"⚡ Threads: {num_threads}, Retries: {retry_max_attempts}")
        print("🚀 Starting synchronization...")
        print("=" * 50)

    # Create and start sync thread
    def sync_worker():
        redirect_enabled = False
        try:
            # Enable print redirection if checkbox is checked
            if redirect_print_checkbox.value:
                enable_print_redirection()
                redirect_enabled = True

            # Call the main sync function
            main_metadata_first()
        except Exception as e:
            with config_output:
                original_print(f"❌ Error during sync: {e}")
        finally:
            # Disable print redirection if it was enabled
            if redirect_enabled:
                disable_print_redirection()

            # Update sync status
            global sync_running
            sync_running = False

            # Only reset button states if sync completed normally (not stopped)
            if not stop_requested:
                header_run_btn.disabled = False
                header_run_btn.description = "🚀 RUN SYNC"
                header_stop_btn.disabled = True

                with config_output:
                    original_print("\n✅ Synchronization completed successfully!")
                    original_print("🔄 Ready for next sync operation.")
            else:
                # If stopped, let the stop_worker handle button states
                with config_output:
                    original_print("\n🛑 Sync process has been stopped.")
                    original_print("⏳ Finalizing stop operation...")

    # Start sync in separate thread
    sync_thread = threading.Thread(target=sync_worker, daemon=True)
    sync_thread.start()

run_btn = ipw.Button(
    description="� Run Synchronization",
    button_style='primary',
    layout=ipw.Layout(width='100%', height='40px')
)
run_btn.on_click(run_sync)

# Output area for configuration and status with scroll
config_output = ipw.Output(layout=ipw.Layout(
    width='100%',
    height='120px',
    border='1px solid #555',
    border_radius='5px',
    padding='8px',
    background_color='#1e1e1e',
    overflow_y='auto'
))



# Connect header buttons to their functions
header_run_btn.on_click(run_sync)
header_stop_btn.on_click(stop_sync)

# Create 2-column layout for better screen utilization
# Row 1: Folder Selection (left) + Comparison Settings (right)
row1 = ipw.HBox([
    folder_section,
    comparison_section
], layout=ipw.Layout(
    width='100%',
    justify_content='space-between',
    align_items='flex-start',
    # margin='2px 0'
))

# Row 2: Performance section (full width)
row2 = performance_section

# Output section (full width at bottom)
output_section = ipw.VBox([
    ipw.HTML(value='<div class="widget-section-title">📊 Status & Output</div>'),
    config_output
], layout=ipw.Layout(
    width='100%',
    margin='4px 0',
    padding='8px',
    border='1px solid #444',
    border_radius='6px',
    background_color='#2d2d2d'
))

credit_section = ipw.HTML(
    value='''
    <div style="text-align: center; margin-top: 10px; padding: 8px;
                background-color: #2d2d2d; border-radius: 6px;
                border: 1px solid #444; color: #ccc; font-size: 12px;">
        💻 Created by: <a href="https://github.com/vannguyen799" target="_blank"
                          style="color: #667eea; text-decoration: none; font-weight: bold;">
                          vannguyen799
                       </a> |
        🔗 GitHub: <a href="https://github.com/vannguyen799" target="_blank"
                      style="color: #667eea; text-decoration: none;">
                      github.com/vannguyen799
                   </a>
    </div>
    ''',
    layout=ipw.Layout(width='100%')
)

# Assemble the complete interface with 2-column layout
main_container.children = [
    title,
    row1,
    row2,
    output_section,
    credit_section
]

# Display the main interface
display(main_container)

# Auto-load saved configuration on startup
try:
    if load_config():
        print("🔄 Auto-loaded saved configuration")
    else:
        print("💡 No saved configuration found - using defaults")
except Exception as e:
    print(f"⚠️  Error auto-loading config: {e}")

Mounted at /content/drive


💡 No saved configuration found - using defaults


In [2]:
# @title def
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import threading
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
import hashlib
from collections import defaultdict

source_folder_id = ''
destination_folder_id = ''
comparison_mode = 'name_only'
num_threads = 4
retry_max_attempts = 5
retry_base_delay = 1.0
retry_max_delay = 60
retry_backoff_factor = 2.0
retry_jitter = True

# Authenticate and create the Drive API service
drive_service = build('drive', 'v3')

# Configuration
source_folder_id = ''
destination_folder_id = ''

# Retry configuration
RETRY_CONFIG = {
    'max_attempts': 5,  # Increased from 3 to 5
    'base_delay': 1,
    'max_delay': 60,    # Increased from 30 to 60
    'backoff_factor': 2,
    'jitter': True
}

# Comparison modes
COMPARISON_MODES = {
    'name_only': 'Compare by file name only',
    'name_and_size': 'Compare by file name and size',
    'name_size_date': 'Compare by file name, size, and modified date',
    'checksum': 'Compare by file name and checksum (slow but accurate)'
}

class FileMetadata:
    """Class to store file metadata"""
    def __init__(self, file_data):
        self.id = file_data.get('id')
        self.name = file_data.get('name')
        self.size = file_data.get('size', '0')
        self.modified_time = file_data.get('modifiedTime')
        self.mime_type = file_data.get('mimeType')
        self.md5_checksum = file_data.get('md5Checksum')
        self.parents = file_data.get('parents', [])
        self.is_folder = self.mime_type == 'application/vnd.google-apps.folder'

    def __repr__(self):
        return f"FileMetadata(name='{self.name}', size={self.size}, is_folder={self.is_folder})"

class MetadataReader:
    """Class to read all metadata first"""

    def __init__(self, comparison_mode='name_only'):
        self.comparison_mode = comparison_mode
        self.drive_service = build('drive', 'v3')

    def _calculate_delay(self, attempt):
        """Calculate delay with exponential backoff and jitter"""
        delay = min(
            RETRY_CONFIG['base_delay'] * (RETRY_CONFIG['backoff_factor'] ** attempt),
            RETRY_CONFIG['max_delay']
        )

        if RETRY_CONFIG['jitter']:
            jitter = delay * 0.25 * (random.random() * 2 - 1)
            delay = max(0.1, delay + jitter)

        return delay

    def _is_retryable_error(self, error):
        """Check if error is retryable"""
        # Handle HTTP errors
        if hasattr(error, 'resp'):
            status_code = int(error.resp.status)
            retryable_codes = [429, 500, 502, 503, 504]
            return status_code in retryable_codes

        # Handle network/connection errors
        error_str = str(error).lower()
        retryable_patterns = [
            'transport endpoint is not connected',
            'broken pipe',
            'bad file descriptor',
            'connection reset',
            'connection refused',
            'timeout',
            'network is unreachable',
            'temporary failure',
            'connection aborted',
            'socket error',
            'ssl error',
            'read timeout',
            'connection timeout',
            'errno 9',
            'errno 32',
            'errno 107'
        ]

        for pattern in retryable_patterns:
            if pattern in error_str:
                return True

        return False

    def read_folder_structure(self, folder_id, folder_name="Root", level=0):
        """
        Recursively read entire folder structure and metadata
        Returns: dict with complete folder tree and file metadata
        """
        print("  " * level + f"📂 Reading: {folder_name}")

        folder_data = {
            'id': folder_id,
            'name': folder_name,
            'level': level,
            'files': {},  # file_name -> FileMetadata
            'folders': {},  # folder_name -> folder_data
            'total_files': 0,
            'total_size': 0
        }

        # Get all items in this folder
        all_items = self._get_all_items_in_folder(folder_id)

        if not all_items:
            return folder_data

        # Separate files and folders
        files = [item for item in all_items if not item['mimeType'] == 'application/vnd.google-apps.folder']
        folders = [item for item in all_items if item['mimeType'] == 'application/vnd.google-apps.folder']

        # Process files
        for file_data in files:
            file_meta = FileMetadata(file_data)
            folder_data['files'][file_meta.name] = file_meta
            folder_data['total_files'] += 1

            # Add to total size (handle files without size like Google Docs)
            try:
                if file_meta.size and file_meta.size.isdigit():
                    folder_data['total_size'] += int(file_meta.size)
            except:
                pass

        print("  " * level + f"   └── Files: {len(files)}")

        # Process subfolders recursively
        for folder_item in folders:
            subfolder_name = folder_item['name']
            subfolder_id = folder_item['id']

            subfolder_data = self.read_folder_structure(
                subfolder_id,
                subfolder_name,
                level + 1
            )

            folder_data['folders'][subfolder_name] = subfolder_data
            folder_data['total_files'] += subfolder_data['total_files']
            folder_data['total_size'] += subfolder_data['total_size']

        if folders:
            print("  " * level + f"   └── Subfolders: {len(folders)}")

        return folder_data

    def _get_all_items_in_folder(self, folder_id, page_size=1000):
        """Get all files and folders from a folder with retry"""
        all_items = []
        page_token = None

        # Determine fields based on comparison mode
        base_fields = "id, name, size, modifiedTime, mimeType, parents"
        if self.comparison_mode == 'checksum':
            base_fields += ", md5Checksum"

        while True:
            for attempt in range(RETRY_CONFIG['max_attempts']):
                try:
                    query = f"'{folder_id}' in parents and trashed = false"

                    results = self.drive_service.files().list(
                        q=query,
                        pageSize=page_size,
                        pageToken=page_token,
                        fields=f"nextPageToken, files({base_fields})"
                    ).execute()

                    items = results.get('files', [])
                    all_items.extend(items)

                    page_token = results.get('nextPageToken')
                    if not page_token:
                        return all_items

                    break  # Success, break retry loop

                except (HttpError, Exception) as error:
                    if attempt < RETRY_CONFIG['max_attempts'] - 1 and self._is_retryable_error(error):
                        delay = self._calculate_delay(attempt)
                        print(f"⚠️  Retry {attempt + 1}/{RETRY_CONFIG['max_attempts']} for folder listing after {delay:.2f}s - {error}")
                        time.sleep(delay)
                        continue
                    else:
                        print(f"❌ Failed to get folder contents after {attempt + 1} attempts: {error}")
                        return all_items  # Return what we have so far

class StructureComparator:
    """Class to compare source and destination structures"""

    def __init__(self, comparison_mode='name_only'):
        self.comparison_mode = comparison_mode

    def compare_structures(self, source_structure, dest_structure):
        """
        Compare two folder structures and create copy plan
        Returns: dict with copy plan
        """
        print("\n🔍 Comparing folder structures...")

        copy_plan = {
            'folders_to_create': [],  # List of folder paths to create
            'files_to_copy': [],      # List of (source_file, dest_path) tuples
            'files_to_skip': [],      # List of files that already exist and match
            'conflicts': []           # List of files with same name but different content
        }

        self._compare_folder_recursive(
            source_structure,
            dest_structure,
            copy_plan,
            source_path="",
            dest_path=""
        )

        return copy_plan

    def _compare_folder_recursive(self, source_folder, dest_folder, copy_plan, source_path, dest_path):
        """Recursively compare folders"""

        # Compare files in current folder
        for file_name, source_file in source_folder['files'].items():
            source_file_path = f"{source_path}/{file_name}".lstrip('/')

            if file_name in dest_folder['files']:
                # File exists, check if identical
                dest_file = dest_folder['files'][file_name]

                if self._files_are_identical(source_file, dest_file):
                    copy_plan['files_to_skip'].append({
                        'file': source_file,
                        'path': source_file_path,
                        'reason': f'Identical file exists ({self.comparison_mode})'
                    })
                else:
                    copy_plan['conflicts'].append({
                        'source_file': source_file,
                        'dest_file': dest_file,
                        'path': source_file_path,
                        'reason': f'File exists but differs ({self.comparison_mode})'
                    })
            else:
                # File doesn't exist, needs to be copied
                copy_plan['files_to_copy'].append({
                    'file': source_file,
                    'source_path': source_file_path,
                    'dest_folder_id': dest_folder['id']
                })

        # Compare subfolders
        for folder_name, source_subfolder in source_folder['folders'].items():
            new_source_path = f"{source_path}/{folder_name}".lstrip('/')

            if folder_name in dest_folder['folders']:
                # Folder exists, compare recursively
                dest_subfolder = dest_folder['folders'][folder_name]
                self._compare_folder_recursive(
                    source_subfolder,
                    dest_subfolder,
                    copy_plan,
                    new_source_path,
                    f"{dest_path}/{folder_name}".lstrip('/')
                )
            else:
                # Folder doesn't exist, needs to be created
                copy_plan['folders_to_create'].append({
                    'folder_name': folder_name,
                    'source_folder': source_subfolder,
                    'parent_folder_id': dest_folder['id'],
                    'path': new_source_path
                })

                # All files in this folder and subfolders need to be copied
                self._add_all_files_to_copy_plan(source_subfolder, copy_plan, new_source_path)

    def _add_all_files_to_copy_plan(self, folder, copy_plan, folder_path):
        """Add all files in folder and subfolders to copy plan"""

        # Add all files in current folder
        for file_name, file_meta in folder['files'].items():
            file_path = f"{folder_path}/{file_name}".lstrip('/')
            copy_plan['files_to_copy'].append({
                'file': file_meta,
                'source_path': file_path,
                'dest_folder_id': None  # Will be set when folder is created
            })

        # Recursively add files from subfolders
        for subfolder_name, subfolder in folder['folders'].items():
            subfolder_path = f"{folder_path}/{subfolder_name}".lstrip('/')
            copy_plan['folders_to_create'].append({
                'folder_name': subfolder_name,
                'source_folder': subfolder,
                'parent_folder_id': None,  # Will be set when parent is created
                'path': subfolder_path
            })
            self._add_all_files_to_copy_plan(subfolder, copy_plan, subfolder_path)

    def _files_are_identical(self, source_file, dest_file):
        """Compare files based on comparison mode"""

        if self.comparison_mode == 'name_only':
            return source_file.name == dest_file.name

        elif self.comparison_mode == 'name_and_size':
            return (source_file.name == dest_file.name and
                   source_file.size == dest_file.size)

        elif self.comparison_mode == 'name_size_date':
            if not (source_file.name == dest_file.name and
                   source_file.size == dest_file.size):
                return False

            # Compare modified time (within 1 second tolerance)
            try:
                source_time = datetime.fromisoformat(source_file.modified_time.replace('Z', '+00:00'))
                dest_time = datetime.fromisoformat(dest_file.modified_time.replace('Z', '+00:00'))

                time_diff = abs((source_time - dest_time).total_seconds())
                return time_diff <= 1
            except (AttributeError, ValueError):
                return True  # If can't parse dates, assume they match

        elif self.comparison_mode == 'checksum':
            if source_file.name != dest_file.name:
                return False

            # Compare checksums if available
            if source_file.md5_checksum and dest_file.md5_checksum:
                return source_file.md5_checksum == dest_file.md5_checksum
            else:
                # Fall back to size comparison for files without checksum
                return source_file.size == dest_file.size

        return False

class SmartCopier:
    """Class to execute the copy plan"""

    def __init__(self):
        self.drive_service = build('drive', 'v3')
        self.folder_id_cache = {}  # Cache for created folder IDs

    def _is_retryable_error(self, error):
        """Check if error is retryable"""
        # Handle HTTP errors
        if hasattr(error, 'resp'):
            status_code = int(error.resp.status)
            retryable_codes = [429, 500, 502, 503, 504]
            return status_code in retryable_codes

        # Handle network/connection errors
        error_str = str(error).lower()
        retryable_patterns = [
            'transport endpoint is not connected',
            'broken pipe',
            'bad file descriptor',
            'connection reset',
            'connection refused',
            'timeout',
            'network is unreachable',
            'temporary failure',
            'connection aborted',
            'socket error',
            'ssl error',
            'read timeout',
            'connection timeout',
            'errno 9',
            'errno 32',
            'errno 107'
        ]

        for pattern in retryable_patterns:
            if pattern in error_str:
                return True

        return False

    def execute_copy_plan(self, copy_plan, num_threads=4):
        """Execute the copy plan with multi-threading"""

        print(f"\n📋 COPY PLAN SUMMARY:")
        print(f"   📁 Folders to create: {len(copy_plan['folders_to_create'])}")
        print(f"   📄 Files to copy: {len(copy_plan['files_to_copy'])}")
        print(f"   ⏭️  Files to skip: {len(copy_plan['files_to_skip'])}")
        print(f"   ⚠️  Conflicts: {len(copy_plan['conflicts'])}")

        if copy_plan['conflicts']:
            print(f"\n⚠️  CONFLICTS DETECTED:")
            for conflict in copy_plan['conflicts'][:5]:  # Show first 5
                print(f"   • {conflict['path']} - {conflict['reason']}")
            if len(copy_plan['conflicts']) > 5:
                print(f"   ... and {len(copy_plan['conflicts']) - 5} more")

            # Automatically skip conflicted files (no user input required)
            print(f"\n📋 Automatically skipping {len(copy_plan['conflicts'])} conflicted files")
            print("   (Files with same name but different content will be skipped)")
            # Conflicts remain in the conflicts list and are not copied

        if not copy_plan['files_to_copy'] and not copy_plan['folders_to_create']:
            print("🎉 Nothing to copy! All files are already up to date.")
            return

        print(f"\n🚀 Starting copy process...")
        start_time = time.time()

        # Step 1: Create folders first
        if copy_plan['folders_to_create']:
            print(f"\n📁 Creating {len(copy_plan['folders_to_create'])} folders...")
            self._create_folders(copy_plan['folders_to_create'])

        # Step 2: Copy files
        if copy_plan['files_to_copy']:
            print(f"\n📄 Copying {len(copy_plan['files_to_copy'])} files using {num_threads} threads...")
            self._copy_files_threaded(copy_plan['files_to_copy'], num_threads)

        total_time = time.time() - start_time
        print(f"\n✅ Copy completed in {total_time:.2f} seconds!")

    def _create_folders(self, folders_to_create):
        """Create folders in correct order (parents first)"""

        # Sort by path depth to create parents first
        folders_to_create.sort(key=lambda x: x['path'].count('/'))

        for folder_info in folders_to_create:
            folder_name = folder_info['folder_name']
            parent_id = folder_info['parent_folder_id']
            path = folder_info['path']

            # Resolve parent ID if not set
            if parent_id is None:
                parent_path = '/'.join(path.split('/')[:-1])
                parent_id = self.folder_id_cache.get(parent_path, destination_folder_id)

            for attempt in range(RETRY_CONFIG['max_attempts']):
                try:
                    print(f"📁 Creating folder: {path}")

                    folder_metadata = {
                        'name': folder_name,
                        'mimeType': 'application/vnd.google-apps.folder',
                        'parents': [parent_id]
                    }

                    created_folder = self.drive_service.files().create(
                        body=folder_metadata,
                        fields='id'
                    ).execute()

                    # Cache the folder ID for future reference
                    self.folder_id_cache[path] = created_folder['id']

                    print(f"✅ Created: {path}")
                    time.sleep(0.1)  # Small delay to avoid rate limiting
                    break  # Success, exit retry loop

                except (HttpError, Exception) as error:
                    if attempt < RETRY_CONFIG['max_attempts'] - 1 and self._is_retryable_error(error):
                        delay = self._calculate_delay(attempt)
                        print(f"⚠️  Retry {attempt + 1} for folder '{path}' after {delay:.2f}s - {error}")
                        time.sleep(delay)
                        continue
                    else:
                        print(f"❌ Failed to create folder '{path}' after {attempt + 1} attempts: {error}")
                        break

    def _copy_files_threaded(self, files_to_copy, num_threads):
        """Copy files using multi-threading"""

        # Resolve destination folder IDs
        for file_info in files_to_copy:
            if file_info['dest_folder_id'] is None:
                # Resolve from path
                file_path = file_info['source_path']
                folder_path = '/'.join(file_path.split('/')[:-1])
                file_info['dest_folder_id'] = self.folder_id_cache.get(folder_path, destination_folder_id)

        results = {
            'copied': [],
            'failed': []
        }

        # Prepare arguments for threading
        copy_args = [(file_info, i) for i, file_info in enumerate(files_to_copy, 1)]

        # Process files with thread pool
        try:
            with ThreadPoolExecutor(max_workers=num_threads) as executor:
                future_to_file = {
                    executor.submit(self._copy_single_file, args): args[0]
                    for args in copy_args
                }

                completed_count = 0
                total_files = len(copy_args)

                for future in as_completed(future_to_file):
                    try:
                        result = future.result()
                        completed_count += 1

                        if result['status'] == 'success':
                            results['copied'].append(result)
                        else:
                            results['failed'].append(result)

                        # Progress update every 10 files
                        if completed_count % 10 == 0 or completed_count == total_files:
                            print(f"📊 Progress: {completed_count}/{total_files} files processed")

                    except Exception as e:
                        print(f"❌ Unexpected error processing file: {e}")
                        results['failed'].append({
                            'status': 'failed',
                            'file_name': 'Unknown',
                            'thread_id': 0,
                            'error': str(e)
                        })

        except Exception as e:
            print(f"❌ Critical error in thread pool: {e}")
            return

        print(f"\n📊 Copy Results:")
        print(f"   ✅ Copied: {len(results['copied'])}")
        print(f"   ❌ Failed: {len(results['failed'])}")

        if results['failed']:
            print(f"\n❌ Failed files:")
            for failed in results['failed'][:5]:
                print(f"   • {failed['file_name']} - {failed.get('error', 'Unknown error')}")

    def _copy_single_file(self, args):
        """Copy a single file with retry"""
        file_info, thread_id = args
        file_meta = file_info['file']
        dest_folder_id = file_info['dest_folder_id']

        for attempt in range(RETRY_CONFIG['max_attempts']):
            try:
                print(f"📄 [T{thread_id:02d}] Copying: {file_meta.name}")

                copy_start = time.time()
                copied_file = self.drive_service.files().copy(
                    fileId=file_meta.id,
                    body={'parents': [dest_folder_id]}
                ).execute()
                copy_time = time.time() - copy_start

                print(f"✅ [T{thread_id:02d}] Copied: {file_meta.name} ({copy_time:.2f}s)")
                time.sleep(0.1)  # Small delay to prevent rate limiting

                return {
                    'status': 'success',
                    'file_name': file_meta.name,
                    'thread_id': thread_id,
                    'time': copy_time,
                    'new_file_id': copied_file.get('id')
                }

            except (HttpError, Exception) as error:
                if attempt < RETRY_CONFIG['max_attempts'] - 1 and self._is_retryable_error(error):
                    delay = self._calculate_delay(attempt)
                    print(f"⚠️  [T{thread_id:02d}] Retry {attempt + 1} for '{file_meta.name}' after {delay:.2f}s - {error}")
                    time.sleep(delay)
                    continue
                else:
                    print(f"❌ [T{thread_id:02d}] Failed: {file_meta.name} - {error}")
                    return {
                        'status': 'failed',
                        'file_name': file_meta.name,
                        'thread_id': thread_id,
                        'error': str(error)
                    }

    def _calculate_delay(self, attempt):
        """Calculate retry delay"""
        delay = min(
            RETRY_CONFIG['base_delay'] * (RETRY_CONFIG['backoff_factor'] ** attempt),
            RETRY_CONFIG['max_delay']
        )

        if RETRY_CONFIG['jitter']:
            jitter = delay * 0.25 * (random.random() * 2 - 1)
            delay = max(0.1, delay + jitter)

        return delay

def format_size(size_bytes):
    """Convert bytes to human readable format"""
    try:
        size = int(size_bytes)
        for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
            if size < 1024:
                return f"{size:.1f} {unit}"
            size /= 1024
        return f"{size:.1f} PB"
    except:
        return "Unknown"

def print_structure_summary(structure, name="Structure"):
    """Print a summary of the folder structure"""
    def count_items(folder):
        files = len(folder['files'])
        folders = len(folder['folders'])
        for subfolder in folder['folders'].values():
            sub_files, sub_folders = count_items(subfolder)
            files += sub_files
            folders += sub_folders
        return files, folders

    total_files, total_folders = count_items(structure)
    total_size = format_size(structure['total_size'])

    print(f"\n📊 {name} Summary:")
    print(f"   📁 Total folders: {total_folders}")
    print(f"   📄 Total files: {total_files}")
    print(f"   💾 Total size: {total_size}")

def main_metadata_first():
    """Main function with metadata-first approach"""
    print("🚀 Google Drive Copy - Metadata-First Approach")
    print("=" * 60)

    # Read configuration from global variables (set by UI widgets)
    global source_folder_id, destination_folder_id

    # Get values from UI widgets
    source_folder_id = src_picker.value.strip()
    destination_folder_id = dst_picker.value.strip()
    comparison_mode = comparison_mode_selector.value
    num_threads = num_threads_input.value
    retry_max_attempts = retry_max_attempts_input.value
    retry_base_delay = retry_base_delay_input.value
    retry_max_delay = retry_max_delay_input.value
    retry_backoff_factor = retry_backoff_factor_input.value
    retry_jitter = retry_jitter_checkbox.value

    # Update RETRY_CONFIG with values from UI
    RETRY_CONFIG['max_attempts'] = retry_max_attempts
    RETRY_CONFIG['base_delay'] = retry_base_delay
    RETRY_CONFIG['max_delay'] = retry_max_delay
    RETRY_CONFIG['backoff_factor'] = retry_backoff_factor
    RETRY_CONFIG['jitter'] = retry_jitter

    # Validate source folder ID - required
    if not source_folder_id:
        print("\n🚫 ERROR: Source folder ID is required!")
        print("=" * 50)
        print("❌ Please provide a valid Google Drive source folder ID.")
        print("💡 You can get the folder ID from the Google Drive URL.")
        print("   Example: 1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms")
        print("=" * 50)
        return  # Stop execution

    # Handle destination folder ID - default to root if empty
    if not destination_folder_id:
        destination_folder_id = 'root'
        print("⚠️  Destination folder ID is empty. Defaulting to Google Drive root folder.")
        print("� Files will be copied to the root of your Google Drive.")

    # Check if source and destination are the same
    if source_folder_id == destination_folder_id:
        print("\n⚠️  WARNING: Source and destination folders are the same!")
        print("   This will create duplicates in the same folder.")
        print("   Consider using a different destination folder.")

    print(f"🔍 Selected comparison mode: {comparison_mode}")
    print(f"⚡ Using {num_threads} threads for copying")
    print(f"🔄 Retry settings: {retry_max_attempts} attempts, {retry_base_delay}s base delay")
    print(f"📂 Source folder ID: {source_folder_id}")
    print(f"📂 Destination folder ID: {destination_folder_id}")
    print("=" * 60)

    # Step 1: Read source structure
    print("📖 Phase 1: Reading source folder structure...")
    metadata_reader = MetadataReader(comparison_mode)

    start_time = time.time()
    source_structure = metadata_reader.read_folder_structure(source_folder_id, "Source")
    read_time = time.time() - start_time

    print_structure_summary(source_structure, "Source")
    print(f"⏱️  Read time: {read_time:.2f} seconds")

    # Step 2: Read destination structure
    print("\n📖 Phase 2: Reading destination folder structure...")
    dest_start_time = time.time()
    dest_structure = metadata_reader.read_folder_structure(destination_folder_id, "Destination")
    dest_read_time = time.time() - dest_start_time

    print_structure_summary(dest_structure, "Destination")
    print(f"⏱️  Read time: {dest_read_time:.2f} seconds")

    # Step 3: Compare structures
    print("\n🔍 Phase 3: Comparing structures...")
    comparator = StructureComparator(comparison_mode)
    copy_plan = comparator.compare_structures(source_structure, dest_structure)

    # Step 4: Execute copy plan
    print("\n🚀 Phase 4: Executing copy plan...")
    copier = SmartCopier()
    copier.execute_copy_plan(copy_plan, num_threads)

    total_time = time.time() - start_time
    print(f"\n🎉 Total process completed in {total_time:.2f} seconds!")
